# Example of DOV search methods based on external WFS

## Use cases

- Filter data within a certain community using its geographic borders
- Filter data within a geographic boundary of a feature in layer "Bekken"
- Filter data within a geographic boundary of a feature in layer "HHZ"
- Filter data within a geographic boundary of a feature in layer "Grondwaterlichamen"

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService

from pydov.search.boring import BoringSearch
from pydov.search.grondwaterfilter import GrondwaterFilterSearch

from pydov.util.location import (
    GmlFilter,
    Within,
)

# Filter data within a certain community using its geographic borders

Load layer:

In [2]:
gemeentegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

Optional get available fields (properties) of a layer:

In [3]:
gemeentegrenzen.get_schema('VRBG:Refgem')['properties']

{'UIDN': 'decimal',
 'OIDN': 'decimal',
 'TERRID': 'decimal',
 'NISCODE': 'string',
 'NAAM': 'string',
 'DATPUBLBS': 'date',
 'NUMAC': 'string'}

Create a custom boundary of an administrative area: 

In [4]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Lievegem')

In [5]:
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

Search all "boringen" within the administrative area: 

In [6]:
bs = BoringSearch()
df = bs.search(
    location=GmlFilter(gemeente_poly, Within),
    return_fields=('pkey_boring', 'gemeente'))

In [7]:
df.groupby('gemeente').size().reset_index(name='counts')

,gemeente,counts
0,Gent,2
1,Knesselare,2
2,Lievegem,12
3,Lovendegem,110
4,Nevele,1
5,Veurne,1
6,Waarschoot,57
7,Zomergem,130


# Filter data within a geographic boundary of a feature in layer "Bekken"

Load layer:

In [8]:
bekkens = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VHAZones/wfs',
    version='1.1.0'
)

Optional list available layers:

In [9]:
list(bekkens.contents)

['VHAZones:Bekken', 'VHAZones:Deelbekken', 'VHAZones:Vhazone']

Optional get available fields (properties) of a layer:

In [10]:
bekkens.get_schema('VHAZones:Bekken')['properties']

{'UIDN': 'decimal',
 'OIDN': 'decimal',
 'BEKNR': 'short',
 'BEKNAAM': 'string',
 'STRMGEB': 'string'}

Get distinct values of a field:

In [11]:
tree = etree.fromstring(bekkens.getfeature('VHAZones:Bekken', propertyname='BEKNAAM').read().encode("utf8"))
set((i.text for i in tree.findall('.//{%s}BEKNAAM' % tree.nsmap['VHAZones'])))

{'Bekken Brugse polders',
 'Bekken Gentse kanalen',
 'Beneden-Scheldebekken',
 'Boven-Scheldebekken',
 'Demerbekken',
 'Denderbekken',
 'Dijlebekken',
 'Ijzerbekken',
 'Leiebekken',
 'Maasbekken',
 'Netebekken'}

Build filter:

In [12]:
naam_filter = PropertyIsEqualTo(propertyname='BEKNAAM', literal='Bekken Brugse polders')

Get features matching filter

In [13]:
bekken_poly = bekkens.getfeature(
    typename='VHAZones:Bekken',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

Get DOV data based on feature geometry

In [14]:
filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    max_features = 100,
    location=GmlFilter(bekken_poly, Within),
    return_fields=('pkey_filter', 'x', 'y')
)

Show on map:

In [15]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [16]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [17]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_filter'][loc]).add_to(marker_cluster)
fmap

# Filter data within a geographic boundary of a feature in layer "HHZ"

In [18]:
hhz = WebFeatureService(
    'https://www.dov.vlaanderen.be/geoserver/wfs',
    version='1.1.0')

todo hhz.get_schema("gw_varia:hhz")

# Filter data within a geographic boundary of a feature in layer "Grondwaterlichamen"

In [19]:
gwl = WebFeatureService(
    'https://www.dov.vlaanderen.be/geoserver/wfs',
    version='1.1.0')

In [20]:
gwl.get_schema("gw_beleid:gwlichamen")

{'properties': {'id': 'long',
  'grondwaterlichaam_afkorting': 'string',
  'grondwaterlichaam': 'string',
  'grondwatersysteem': 'string',
  'EU_code': 'string',
  'stroomgebiedsdistrict': 'string',
  'horizonten': 'string'},
 'geometry': '3D MultiPolygon',
 'geometry_column': 'geom'}

In [21]:
grondwaterlichaam_filter = PropertyIsEqualTo(propertyname='grondwaterlichaam_afkorting', literal='BLKS_0600_GWL_1')

In [22]:
grondwaterlichaam_poly = gwl.getfeature(
    typename='gw_beleid:gwlichamen',
    filter=etree.tostring(grondwaterlichaam_filter.toXML()).decode("utf8")).read()

In [23]:
gemeentegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

In [24]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Hoegaarden')

In [25]:
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

In [26]:
from owslib.fes import And

In [27]:
filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    location=And([GmlFilter(grondwaterlichaam_poly, Within), GmlFilter(gemeente_poly, Within)]),
    return_fields=('pkey_filter', 'x', 'y', 'aquifer_code', 'gemeente', 'grondwaterlichaam_code'))

[000/047] ccccccccccccccccccccccccccccccccccccccccccccccc


In [28]:
df

,pkey_filter,x,y,aquifer_code,gemeente,grondwaterlichaam_code
0,https://www.dov.vlaanderen.be/data/filter/2003...,188318.6600,160810.980000,0620,Hoegaarden,BLKS_0600_GWL_1
1,https://www.dov.vlaanderen.be/data/filter/2003...,188318.6600,160810.980000,0620,Hoegaarden,BLKS_0600_GWL_1
2,https://www.dov.vlaanderen.be/data/filter/2003...,188318.6600,160810.980000,1010,Hoegaarden,BLKS_1000_GWL_1S
3,https://www.dov.vlaanderen.be/data/filter/1996...,183090.0000,165316.000000,0620,Hoegaarden,NaN
4,https://www.dov.vlaanderen.be/data/filter/1900...,182178.0000,165077.000000,1010,Hoegaarden,NaN
5,https://www.dov.vlaanderen.be/data/filter/2000...,185319.0000,165155.000000,1000,Hoegaarden,BLKS_1000_GWL_2S
6,https://www.dov.vlaanderen.be/data/filter/1950...,182992.0000,165091.000000,1010,Hoegaarden,NaN
7,https://www.dov.vlaanderen.be/data/filter/2003...,185891.3600,165155.100000,0600,Hoegaarden,BLKS_0600_GWL_1
8,https://www.dov.vlaanderen.be/data/filter/2003...,185891.3600,165155.100000,0600,Hoegaarden,BLKS_0600_GWL_1
9,https://www.dov.vlaanderen.be/data/filter/2003...,185891.3600,165155.100000,0600,Hoegaarden,BLKS_0600_GWL_1
